# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
pip install hvplot

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,east london,-33.0153,27.9116,57.18,58,99,5.59,ZA,1686006453
1,1,grytviken,-54.2811,-36.5092,36.25,97,100,4.56,GS,1686006453
2,2,bredasdorp,-34.5322,20.0403,47.82,76,61,3.78,ZA,1686006453
3,3,nadi,-17.8000,177.4167,82.40,65,75,3.44,FJ,1686006454
4,4,kananga,-5.8958,22.4178,70.29,87,56,2.77,CD,1686006454


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
pip install folium


  Using cached folium-0.14.0-py2.py3-none-any.whl (102 kB)
  Using cached branca-0.6.0-py3-none-any.whl (24 kB)



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import folium

# Create a base map using folium
map_plot = folium.Map(location=[city_data_df["Lat"].mean(), city_data_df["Lng"].mean()], zoom_start=4)

# Add markers for each city with humidity information
for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row["Lat"], row["Lng"]],
        radius=row["Humidity"] / 10,  # Adjust the size of the circle based on humidity
        popup=f"City: {row['City']}<br>Humidity: {row['Humidity']}",
        fill=True,
        color='blue',
        fill_color='blue',
    ).add_to(map_plot)

# Display the map
map_plot

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
min_temp_s = city_data_df["Max Temp"] > 21
max_temp_s = city_data_df["Max Temp"] < 40
wind_speed_s = city_data_df["Wind Speed"] > 5

In [39]:
print(sum(min_temp_s))
print(sum(max_temp_s))
print(sum(wind_speed_s))

579
26
346


In [40]:
# A max temperature lower than 27 degrees but higher than 21
# Wind speed more than 5 m/s
# Narrow down cities that fit criteria and drop any results with null values
# Max temp
city_temp_range_df = city_data_df[min_temp_s & max_temp_s & wind_speed_s]
city_temp_range_df
# Drop any rows with null values & Display sample data
clean_city_temp_range_df = city_temp_range_df.dropna()
clean_city_temp_range_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,gadzhiyevo,69.2551,33.3362,36.32,97,100,22.01,RU,1686006454
6,6,port-aux-francais,-49.3500,70.2167,37.40,63,52,37.67,TF,1686006454
15,15,ilulissat,69.2167,-51.1000,32.02,74,75,13.80,GL,1686006451
17,17,olonkinbyen,70.9221,-8.7187,31.82,84,57,6.85,SJ,1686006456
20,20,iskateley,67.6803,53.1512,39.85,75,95,23.76,RU,1686006456
29,29,nuuk,64.1835,-51.7216,35.13,99,100,5.70,GL,1686006458
33,33,punta arenas,-53.1500,-70.9167,37.51,93,0,9.22,CL,1686006458
34,34,ushuaia,-54.8000,-68.3000,38.73,95,100,5.19,AR,1686006459
45,45,aasiaat,68.7098,-52.8699,30.96,81,93,10.13,GL,1686006199
70,70,udachny,66.4167,112.4000,34.45,80,92,13.69,RU,1686006465


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:


# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_city_temp_range_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
hotel_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,5,gadzhiyevo,69.2551,33.3362,36.32,97,100,22.01,RU,1686006454,
6,6,port-aux-francais,-49.3500,70.2167,37.40,63,52,37.67,TF,1686006454,
15,15,ilulissat,69.2167,-51.1000,32.02,74,75,13.80,GL,1686006451,
17,17,olonkinbyen,70.9221,-8.7187,31.82,84,57,6.85,SJ,1686006456,
20,20,iskateley,67.6803,53.1512,39.85,75,95,23.76,RU,1686006456,
29,29,nuuk,64.1835,-51.7216,35.13,99,100,5.70,GL,1686006458,
33,33,punta arenas,-53.1500,-70.9167,37.51,93,0,9.22,CL,1686006458,
34,34,ushuaia,-54.8000,-68.3000,38.73,95,100,5.19,AR,1686006459,
45,45,aasiaat,68.7098,-52.8699,30.96,81,93,10.13,GL,1686006199,
70,70,udachny,66.4167,112.4000,34.45,80,92,13.69,RU,1686006465,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [42]:
# Set parameters to search for a hotel
radius = 10000
limit = 1
categories = "accommodation.hotel"

#create a params dictionary
params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key     
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = hotel_df.loc[index, "Lng"] 
    lat = hotel_df.loc[index, "Lat"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    print(name_address.url)
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=caf43acae54a4a34bfcc6d93df8c3191&filter=circle%3A33.3362%2C69.2551%2C10000&bias=proximity%3A33.3362%2C69.2551
gadzhiyevo - nearest hotel: Оазис
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=caf43acae54a4a34bfcc6d93df8c3191&filter=circle%3A70.2167%2C-49.35%2C10000&bias=proximity%3A70.2167%2C-49.35
port-aux-francais - nearest hotel: Keravel
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=caf43acae54a4a34bfcc6d93df8c3191&filter=circle%3A-51.1%2C69.2167%2C10000&bias=proximity%3A-51.1%2C69.2167
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
https://api.geoapify.com/v2/places?categories=accommodation.hotel&limit=1&apiKey=caf43acae54a4a34bfcc6d93df8c3191&filter=circle%3A-8.7187%2C70.9221%2C10000&bias=proximity%3A-8.7187%2C70.9221
olonkinbyen - nearest hotel: No hotel found
https://api.geoapify.com/v2/places?cat

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,5,gadzhiyevo,69.2551,33.3362,36.32,97,100,22.01,RU,1686006454,Оазис
6,6,port-aux-francais,-49.3500,70.2167,37.40,63,52,37.67,TF,1686006454,Keravel
15,15,ilulissat,69.2167,-51.1000,32.02,74,75,13.80,GL,1686006451,Best Western Plus Hotel Ilulissat
17,17,olonkinbyen,70.9221,-8.7187,31.82,84,57,6.85,SJ,1686006456,No hotel found
20,20,iskateley,67.6803,53.1512,39.85,75,95,23.76,RU,1686006456,Авантаж
29,29,nuuk,64.1835,-51.7216,35.13,99,100,5.70,GL,1686006458,Nordbo
33,33,punta arenas,-53.1500,-70.9167,37.51,93,0,9.22,CL,1686006458,Hotel Lacolet
34,34,ushuaia,-54.8000,-68.3000,38.73,95,100,5.19,AR,1686006459,Apart Hotel Aires del Beagle
45,45,aasiaat,68.7098,-52.8699,30.96,81,93,10.13,GL,1686006199,SØMA
70,70,udachny,66.4167,112.4000,34.45,80,92,13.69,RU,1686006465,Вилюй


In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
import folium

# Create a base map using folium
hotel_map_plot = folium.Map(location=[hotel_df["Lat"].mean(), hotel_df["Lng"].mean()], zoom_start=4)

# Add markers for each hotel with city information as popups
for index, row in hotel_df.iterrows():
    folium.Marker(
        location=[row["Lat"], row["Lng"]],
        popup=f"City: {row['City']}<br>Hotel Name: {row['Hotel Name']}",
        icon=folium.Icon(color="blue")
    ).add_to(hotel_map_plot)

# Display the map
hotel_map_plot